In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error




In [2]:

# 1. Load
train = pd.read_csv('train.csv')
val   = pd.read_csv('val.csv')
test  = pd.read_csv('test.csv')

In [3]:
# 1. Compute fill‐values on TRAIN only
mg_mode      = train['MG'].mode()[0]           # mode for categorical MG
lon_median   = train['Longitude'].median()     # median for longitude
mean_cols    = ['Lodging','PlantHeight','SeedSize','Protein','Oil']
mean_values  = train[mean_cols].mean()         # means for plant characteristics

# 2. Fill missing in train/val/test
for df in (train, val, test):
    df['MG']        = df['MG'].fillna(mg_mode)
    df['Longitude'] = df['Longitude'].fillna(lon_median)
    for col in mean_cols:
        df[col]     = df[col].fillna(mean_values[col])

# 3. Define your features

In [4]:
# make sure these are defined at top‐level:
temporal_feats = ['MaxTemp','MinTemp','AvgTemp','AvgHumidity','Precipitation','Radiation']
static_feats   = ['Latitude','Longitude','Row.Spacing']
plant_feats    = ['Lodging','PlantHeight','SeedSize','Protein','Oil']
cluster_feats  = [f'Cluster_{i}' for i in range(40)]

def aggregate_sequences(df, target='Yield', agg_target='mean'):
    agg_dict = {}

    # 1. temporal: mean & std
    for feat in temporal_feats:
        agg_dict[f'{feat}_mean'] = (feat, 'mean')
        agg_dict[f'{feat}_std']  = (feat, 'std')

    # 2. static geography: take first (constant per sequence)
    for feat in static_feats:
        agg_dict[feat] = (feat, 'first')

    # 3. plant features:
    #    - MG (categorical) → mode  
    agg_dict['MG'] = ('MG', lambda x: x.mode().iloc[0])
    #    - Lodging, PlantHeight, SeedSize, Protein, Oil → first
    for feat in plant_feats:
        agg_dict[feat] = (feat, 'first')

    # 4. cluster indicators: proportion of time in each cluster + variability
    for feat in cluster_feats:
        agg_dict[f'{feat}_mean'] = (feat, 'mean')
        agg_dict[f'{feat}_std']  = (feat, 'std')

    # 5. target: mean or final
    if agg_target == 'mean':
        agg_dict[target] = (target, 'mean')
    elif agg_target == 'final':
        agg_dict[target] = (target, lambda x: x.iloc[-1])
    else:
        raise ValueError("agg_target must be 'mean' or 'final'")

    # apply the aggregation
    grouped = df.groupby('TimeSeriesLabel').agg(**agg_dict)
    return grouped.reset_index(drop=True)


In [5]:
train_agg = aggregate_sequences(train, agg_target='mean')
val_agg   = aggregate_sequences(val,   agg_target='mean')
test_agg  = aggregate_sequences(test,  agg_target='mean')

In [6]:
# Prepare feature / target matrices as before, but convert to numpy:
X_train = train_agg.drop('Yield', axis=1).to_numpy()
y_train = train_agg['Yield'].to_numpy()
X_val   = val_agg.drop('Yield',   axis=1).to_numpy()
y_val   = val_agg['Yield'].to_numpy()
X_test  = test_agg.drop('Yield',  axis=1).to_numpy()
y_test  = test_agg['Yield'].to_numpy()



In [7]:
import numpy as np
from tabdpt import TabDPTRegressor
from sklearn.metrics import r2_score, root_mean_squared_error


# Instantiate and train on all ~87k rows:
model = TabDPTRegressor()
model.fit(X_train, y_train)


for name, X_np, y_np in [
        ('Val',  X_val,  y_val),
        ('Test', X_test, y_test)
    ]:
    preds = model.predict(
        X_np,
        n_ensembles=8,                    # trade speed vs. stability
        context_size=len(X_train),        # ≥ number of rows in X_train
    )
    print(f"{name} R²:  {r2_score(y_np, preds):.3f},",
          f"RMSE: {root_mean_squared_error(y_np, preds):.3f}")



  0%|          | 0/8 [00:00<?, ?it/s]c:\Users\mohd7\AppData\Local\miniconda3\envs\tabdpt_env\Lib\site-packages\torch\_dynamo\utils.py:3284: UserWarning: Memory efficient kernel not used because: (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:838.)
  return node.target(*args, **kwargs)
c:\Users\mohd7\AppData\Local\miniconda3\envs\tabdpt_env\Lib\site-packages\torch\_dynamo\utils.py:3284: UserWarning: Memory Efficient attention has been runtime disabled. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen/native/transformers/sdp_utils_cpp.h:548.)
  return node.target(*args, **kwargs)
c:\Users\mohd7\AppData\Local\miniconda3\envs\tabdpt_env\Lib\site-packages\torch\_dynamo\utils.py:3284: UserWarning: Flash attention kernel not used because: (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:840.)
  return 

TorchRuntimeError: Dynamo failed to run FX node with fake tensors: call_function <built-in function scaled_dot_product_attention>(*(FakeTensor(..., device='cuda:0', size=(1, 8, 96864, 96)), FakeTensor(..., device='cuda:0', size=(1, 8, 86101, 96)), FakeTensor(..., device='cuda:0', size=(1, 8, 86101, 96), dtype=torch.bfloat16)), **{}): got RuntimeError('No available kernel. Aborting execution.')

from user code:
   File "C:\Users\mohd7\TabDPT\src\tabdpt\utils.py", line 30, in wrapper
    return func(self, *args, **kwargs)
  File "C:\Users\mohd7\TabDPT\src\tabdpt\model.py", line 72, in forward
    src = layer(src, eval_pos)
  File "C:\Users\mohd7\TabDPT\src\tabdpt\model.py", line 134, in forward
    attn = F.scaled_dot_product_attention(q, k, v).transpose(1, 2)

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"
